```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Some theory

In this notebook we investigate different properties with our numerical schemes related to geostrophic balance. Geostrophic balance represent steady state solutions where the pressure gradients are balanced by the Coriolis forces.

## Geostrophic Balance

The geostrophic equations in rotating shalow water is given by
$$ \frac{\partial u}{\partial t} - fv  = - \frac{1}{\rho} \frac{\partial p}{\partial x}, $$
$$ \frac{\partial v}{\partial t} + fu  = - \frac{1}{\rho} \frac{\partial p}{\partial y}. $$
By the assumption that the vertical velocity is negligible compared to the horizontal velocity, we integrate the equations vertically.
Using hydrostatic pressure $ p = \rho g (H+\eta) + p_{atm}$, we get an expression for the change in momentum as
$$ \frac{\partial hu}{\partial t} =  fhv - gh\frac{\partial \eta}{\partial x}, $$
$$ \frac{\partial hv}{\partial t} = -fhu - gh\frac{\partial \eta}{\partial y}. $$
At geostrophic balance, the steady state solution is described by $\frac{\partial hv}{\partial t} = \frac{\partial hu}{\partial t} = 0$.


*What are the references for the above explanation?*

###### Additional geostrophy equation
The following equation is related to the above, to describe the steady-state of the surface elevation:
$$\frac{\partial \eta}{\partial t} + hu\frac{\partial \eta }{\partial x } + hv \frac{\partial \eta }{\partial y} = 0 $$

## Rossby adjustment
Rossby adjustment is the process where an initial bump (of some sort), $\eta_0$, reaches the geostrophic balance steady-state $\bar{\eta}$. The size of the steady-state bump can be described by gravity $g$, lake-at-rest depth $H$ and Coriolis forces $f$ by the Klein-Gordon equation,
$$ -c_0^2 \nabla^2 \bar{\eta} + f^2 (\bar{\eta} - \eta_0) = 0, $$
or, more commonly (?),
$$  \nabla^2 \bar{\eta} -  \left( \frac{1}{a} \right)^2 (\bar{\eta} - \eta_0) = 0. $$
In this equation, $(1/a)^2$ is the Rossby radius. The constant $c_0^2 = gH$, meaning that $a^2 = gH/f^2$. 
The physical interpretation of the Rossby radius, $a$, is shown in the first figure below the initial imports.

Additionally, the steady-state solution should be reached within a period given by $\approx \pi/f$.


## Topographic Rossby adjustment

Given an initial bump of water above a linear (but non-constant) bottom topography, no bump should be created, but rather a flow similar to comet orbits.

## What to investigate:

- [x] Check Geostrophic Balance
- [x] Check Klein-Gordon
- [_] Wave front reach 
- [_] Stability height (compare cross sections, and compare Rossby radius)

# Setting up the environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

import os, pyopencl, datetime, sys

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, RecursiveCDKLM16, DataOutput, SimWriter, PlotHelper, Common
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [ ]:
## Physical interpretation of the Rossby radius:
def rossbyTheoreticFigure():
    fig = plt.figure(figsize=(4, 3))
    N=100
    x = np.linspace(0, 4, N)
    eta_bar = np.exp(-(2 - x)**2)
    eta_0 = np.zeros_like(x)
    r = (N/2)*0.3
    for i in range(N):   
        if i > (N/2) - r and i < (N/2)+r:
            eta_0[i] = 1.3

    plt.plot(eta_bar, 'b', label="eta_bar")
    plt.plot(eta_0,   'r', label="eta_0")
    ind = int(N/2 +r)
    print ind
    plt.plot([ind], [eta_bar[ind]], 'co', label="a")
    axes = plt.gca()
    axes.set_ylim([-0.3, 3])
    plt.legend()
rossbyTheoreticFigure()

The Rossby radius can be described as "the distance that a particle or wave travels before being significantly affected by the earth's rotation" (http://www.geo.cornell.edu/ocean/p_ocean/ppt_notes/21_KelvinRossbyWaves.pdf, slide 4).

*I'm not sure how to interpret $a$ exactly. Is it the drop from 1.4 to ~0.7? Or is it the height from 0.0 to ~0.7? The fact that these two meassures are the same here is a coincident only. *

## Evaluation of Klein-Gordon and the Geostrophic Balance

Given the equation for the geostrophic balance in Eq (??), and the Klein-Gordon equation in Eq (??), above, we approximate derivatives of the solution to investigate steady-state properties.

In [ ]:
"""
Returns d/dx(0.5*g*h*h) + fhv
"""
def geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells in input arrays
    A = hu*hv/h
    B = hu*hu/h
    C = 0.5*g*h*h
    D = -f*hv
    
    Ay = np.zeros_like(A)
    Bx = np.zeros_like(B)
    Cx = np.zeros_like(C)
    
    Ay[1:-1,:] = (A[:-2,:] - A[2:,:])/(2*dy)
    Bx[:, 1:-1] = (B[:,:-2] - B[:,2:])/(2*dx)
    Cx[:, 1:-1] = (C[:,:-2] - C[:,2:])/(2*dx)
    
    #geoBalance = (Cx )/D - 1
    geoBalance = Cx - D
    #geoBalance = (Ay + Bx + Cx )/D - 1
    #geoBalance = (Ay + Bx + Cx - D)/(0.5*(Ay + Bx + Cx + D))
    return geoBalance

def geostrophicBalanceStaggered(eta, H, hu_s, hv_s, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
       
    h = eta + H
    hu = 0.5*(hu_s[:, :-1] + hu_s[:, 1:])
    hv = 0.5*(hv_s[:-1, :] + hv_s[1:, :])
    return geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g )

In [ ]:
#cdklm_h, cdklm_hu, cdklm_hv = sim.download()
def kleinGordonBalance(eta, H, eta_0, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
    c = np.sqrt(g*H)
    a = c/f
    constant = 1/(a*a)
    print "a: , const: ", a, constant
    print "dx, dy:", dx, dy
    print "max eta-eta_0: ", np.max(eta - eta_0)
    
    eta_xx = np.zeros_like(eta)
    eta_yy = np.zeros_like(eta)
    laplace_eta = np.zeros_like(eta) 
        
    eta_yy[1:-1, :] = (eta[:-2,:] -2*eta[1:-1,:] + eta[2:, :])/(dy*dy)
    eta_xx[:, 1:-1] = (eta[:, :-2] - 2*eta[:, 1:-1] + eta[:, 2:])/(dx*dx)
    laplace_eta = eta_xx + eta_yy
    print "max Laplacean: ", np.max(laplace_eta)
    
    kg = laplace_eta - constant*(eta - eta_0)
    return kg
    
    

# Rossby adjustment on flat bottom

The area of interest is simply the neighbouring region around the initial bump (where  $\bar{\eta} \neq 0$ at steady state). We will therefore use a domain that is large enough to not generate reflections within the area of interest during the simulation time. Ideally, the boundary conditions should be open, but we will here use wall boundary conditions.

Initial conditions are created from a step function
$$h(x,y) = \begin{cases} H_0 + H_{step} &\mathrm{if}\; (x-x_c)^2 + (y - y_c)^2 \leq r^2 \; \mathrm{for\;some} \;r \\ H_0 &\mathrm{otherwise} \end{cases}$$

In [ ]:
# Common parameters from Kai

nx = 800
ny = 1000

dx = 50000
dy = 50000

dt = 100
g = 9.81
f = 1.2e-4
r = 0.0
H0 = 1000.0
A = 0.0 # A is diffusion coefficient multiplied by depth.

wind = Common.WindStressParams(type=99)
#bcSettings = 1
boundaryConditions = Common.BoundaryConditions()

# Rossby adjustment parameters:
c0_squared = g*H0
a_squared = g*H0/(f*f)

T_steady_state = np.pi/f
print T_steady_state

T = 300
sub_T = 1000.0

interestingDomain = 250

paramSet = "Kai"
msg = "Using variables from " + paramSet

# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

In [ ]:
# Common parameters from Haavard

nx = 200
ny = 200

dx = 200.0
dy = 200.0

dt = 0.09
g = 9.81
f = 0.01
r = 0.0

H0 = 60.0
H_step = 0.3

A = 1 # Numerical instability parameter for CTCS

wind = Common.WindStressParams(type=99)
#bcSettings = 1
boundaryConditions = Common.BoundaryConditions()

# Rossby adjustment parameters:
c0_squared = g*H0
a_squared = g*H0/(f*f)

T_steady_state = np.pi/f
print T_steady_state

T = 77
sub_T = 10.0

interestingDomain = 50

paramSet = "Haavard"
msg = "Using variables from " + paramSet
# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

##### Creating initial conditions

Staggered and unstaggered grids are placed so that cell centers are on the same place. Keep therefore in mind that the velocities are defined at different positions for the two different grid types!

In [ ]:
def initialConditions(eta0, nx, ny, dx, dy, halo_x, halo_y):
    if paramSet == "Haavard":
        ghosts = [halo_y, halo_x, halo_y, halo_x]
        addCentralBump(eta0, nx, ny, dx/5, dy/5, ghosts)
        #addCentralDamBreakStep(ctcs_eta0, nx, ny, dx, dy, H_step, ghosts)
        #addCentralDamBreakSmooth(ctcs_eta0, nx, ny, dx, dy, H_step, ghosts)
        return
    print "Making initial conditions"
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    y_center1 = dy*(ny-100)/2.0
    y_center2 = dy*(ny+100)/2.0

    for j in range(-halo_y, ny+halo_y):
        for i in range(-halo_x, nx+halo_x):
            x = dx*i - x_center 
            y = dy*j - y_center 
            y1 = dy*j - y_center1
            y2 = dy*j - y_center2

            # Initial smooth step
            inirad = np.sqrt(x**2 + y**2)
            inirad1 = np.sqrt(x**2 + y1**2)
            inirad2 = np.sqrt(x**2 + y2**2)
            L = 15*dx
            D = 50*dx
            etaamp = 0.2
            
            # Add the the initial condition to the provided array
            eta0[j+1, i+1] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            

## Executing simulators

In [ ]:
#Centered in time, centered in space

ghosts = [1,1,1,1]

ctcs_h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * H0;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

ctcs_eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
ctcs_u0 = np.zeros((ny+2, nx+1+2), dtype=np.float32, order='C');
ctcs_v0 = np.zeros((ny+1+2, nx+2), dtype=np.float32, order='C');

initialConditions(ctcs_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#Initialize simulator
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()
reload(CTCS)
ctcs_sim = CTCS.CTCS(cl_ctx, \
                     ctcs_h0, ctcs_eta0, ctcs_u0, ctcs_v0, \
                     nx, ny, \
                     dx, dy, dt, \
                     g, f, r, A, \
                     wind_stress=wind, \
                     boundary_conditions=boundaryConditions )


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, ctcs_eta0[1:-1, 1:-1], ctcs_u0[1:-1, :], ctcs_v0[:, 1:-1])

print "CTCS\n" + msg
def animate(i):
    if (i>0):
        t = ctcs_sim.step(sub_T)
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1 + H0)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#Coriolis well balanced reconstruction scheme

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
   
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

cdklm_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
cdklm_u0 = np.zeros(dataShape, dtype=np.float32, order='C');
cdklm_v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

initialConditions(cdklm_h0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#Initialize simulator
if 'cdklm_sim' in globals():
    cdklm_sim.cleanUp()
reload(CDKLM16)
cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                            cdklm_h0, cdklm_u0, cdklm_v0, \
                            Bi, \
                            nx, ny, \
                            dx, dy, dt, \
                            g, f, r, \
                            wind_stress=wind, \
                            boundary_conditions=boundaryConditions, \
                            reportGeostrophicEquilibrium=True)



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                cdklm_h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - H0, 
                                cdklm_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                cdklm_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

Kx = np.zeros(T)
Ly = np.zeros(T)
uxpvy = np.zeros(T)

print "CDKLM\n" + msg
def animate(i):
    if (i>0):
        t = cdklm_sim.step(sub_T)
    else:
        t = 0.0
    h1, u1, v1 = cdklm_sim.download()
    uxpvy[i], Kx[i], Ly[i] = cdklm_sim.downloadGeoEqNorm()
    
    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - H0, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
# Kurgonov-Petrove scheme from 2007

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
   
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

kp07_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
kp07_u0 = np.zeros(dataShape, dtype=np.float32, order='C');
kp07_v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
#linearBathymetryY(Bi, nx, ny, dx, dy, ghosts, 5, 50)

initialConditions(kp07_h0, nx, ny, dx, dy, ghosts[1], ghosts[0])

#Initialize simulator
if 'kp07_sim' in globals():
    kp07_sim.cleanUp()
reload(KP07)
kp07_sim = KP07.KP07(cl_ctx, \
                     kp07_h0, Bi, kp07_u0, kp07_v0, \
                     nx, ny, \
                     dx, dy, dt, \
                     g, f, r, \
                     wind_stress=wind, \
                     boundary_conditions=boundaryConditions)



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                kp07_h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - H0, 
                                kp07_u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                kp07_v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])


print "KP07\n" + msg
def animate(i):
    if (i>0):
        t = kp07_sim.step(sub_T)
    else:
        t = 0.0
    h1, u1, v1 = kp07_sim.download()
    
    plotter.plot(h1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - H0, 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(h1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

## Results

In [ ]:
cdklm_h, cdklm_hu, cdklm_hv = cdklm_sim.download()

fig = plt.figure(figsize=(10, 8))
plt.subplot(1,2,1)    
gb_CDKLM = geostrophicBalance(cdklm_h[2:-2, 2:-2], \
                              cdklm_hu[2:-2, 2:-2], \
                              cdklm_hv[2:-2, 2:-2], \
                              nx, ny, \
                              dx, dy, \
                              f, g)
plt.imshow(gb_CDKLM[1:-1, 1:-1], interpolation="none")#, vmin=-1, vmax =1)
plt.title("CDKLM - Geostrophic Balance in x-direction")
plt.colorbar()

plt.subplot(1,2,2)
gb_CDKLM2 = geostrophicBalance(cdklm_h[2:-2, 2:-2].T, \
                              cdklm_hv[2:-2, 2:-2].T, \
                               cdklm_hu[2:-2, 2:-2].T, \
                               nx, ny, \
                              dx, dy, \
                              -f, g)
plt.imshow(gb_CDKLM2[1:-1, 1:-1].T, interpolation="none")#, vmin=-1, vmax =1)
plt.title("CDKLM - Geostrophic Balance in y-direction")
plt.colorbar()

fig = plt.figure(figsize=(5, 4))
cdklm_eta_0 = cdklm_h0 - H0
KG_CDKLM = kleinGordonBalance(cdklm_h[2:-2, 2:-2] - H0, H0, \
                              cdklm_eta_0[2:-2, 2:-2], \
                              nx, ny, \
                             dx, dy, \
                              f, g)
plt.imshow(KG_CDKLM)
plt.title("CDKLM - Klein-Gordon balance")
plt.colorbar()

In [ ]:
ctcs_eta, ctcs_hu, ctcs_hv = ctcs_sim.download()

fig = plt.figure(figsize=(10, 8))
plt.subplot(1,2,1)    
gb_CTCS = geostrophicBalanceStaggered(ctcs_eta[1:-1, 1:-1], H0, \
                               ctcs_hu[1:-1, 1:-1], \
                               ctcs_hv[1:-1, 1:-1], \
                               nx, ny, \
                               dx, dy, \
                               f, g)

plt.imshow(gb_CTCS[1:-1, 1:-1], interpolation="none")#, vmin=-1, vmax =1)
plt.title("CTCS - Geostrophic Balance in x-direction")
plt.colorbar()

plt.subplot(1,2,2)
gb_CTCS2 = geostrophicBalanceStaggered(ctcs_eta[1:-1, 1:-1].T, H0, \
                                       ctcs_hv[1:-1, 1:-1].T, \
                                       ctcs_hu[1:-1, 1:-1].T, \
                                       nx, ny, \
                                       dx, dy, \
                                       -f, g)
plt.imshow(gb_CTCS2[1:-1, 1:-1], interpolation="none")#, vmin=-1, vmax =1)

plt.title("CTCS - Geostrophic Balance in y-direction")
plt.colorbar()

fig = plt.figure(figsize=(5, 4))
KG_CTCS = kleinGordonBalance(ctcs_eta[1:-1, 1:-1], H0, \
                             ctcs_eta0[1:-1, 1:-1], \
                             nx, ny, \
                             dx, dy, \
                             f, g)
plt.imshow(KG_CTCS)
plt.title("CTCS - Klein-Gordon balance")
plt.colorbar()

In [ ]:
kp07_h, kp07_hu, kp07_hv = kp07_sim.download()

fig = plt.figure(figsize=(10, 8))
plt.subplot(1,2,1)    
gb_KP07 = geostrophicBalance(kp07_h[2:-2, 2:-2], \
                             kp07_hu[2:-2, 2:-2], \
                             kp07_hv[2:-2, 2:-2], \
                             nx, ny, \
                             dx, dy, \
                             f, g)
plt.imshow(gb_KP07[1:-1, 1:-1], interpolation="none")#, vmin=-1, vmax =1)
plt.title("KP07 - Geostrophic Balance in x-direction")
plt.colorbar()

plt.subplot(1,2,2)
gb_KP072 = geostrophicBalance(kp07_h[2:-2, 2:-2].T, \
                              kp07_hv[2:-2, 2:-2].T, \
                              kp07_hu[2:-2, 2:-2].T, \
                              nx, ny, \
                              dx, dy, \
                              -f, g)
plt.imshow(gb_KP072[1:-1, 1:-1].T, interpolation="none")#, vmin=-1, vmax =1)
plt.title("KP072 - Geostrophic Balance in y-direction")
plt.colorbar()

fig = plt.figure(figsize=(5, 4))
kp07_eta_0 = kp07_h0 - H0
KG_KP07 = kleinGordonBalance(kp07_h[2:-2, 2:-2] - H0, H0, \
                              kp07_eta_0[2:-2, 2:-2], \
                              nx, ny, \
                             dx, dy, \
                              f, g)
plt.imshow(KG_KP07)
plt.title("KP07 - Klein-Gordon balance")
plt.colorbar()

In [ ]:
# Comparison of bump across the different methods:

fig = plt.figure(figsize=(10, 9))
plt.subplot(2,1,1)
plt.title("$\eta$ at geostrophic balance")
plt.plot(ctcs_eta0[ny/2+1, nx/2+1-interestingDomain:nx/2+1+interestingDomain], 'k', label="init")
plt.plot(ctcs_eta[ny/2+1, nx/2+1-interestingDomain:nx/2+1+interestingDomain], 'b-.', label="CTCS")
plt.plot(cdklm_h[ny/2+2, nx/2+2-interestingDomain:nx/2+2+interestingDomain] - H0, 'r--', label="CDKLM")
plt.plot(kp07_h[ny/2+2, nx/2+2-interestingDomain:nx/2+2+interestingDomain] - H0, 'g:', label="KP07")
plt.ylim((-0.2, 0.3))
plt.legend()

plt.subplot(2,1,2)
plt.title("$\eta$ at geostrophic balance - global ")
plt.plot(ctcs_eta0[ny/2+1, 1:-1], 'k', label="init")
plt.plot(ctcs_eta[ny/2+1, 1:-1], 'b-.', label="CTCS")
plt.plot(cdklm_h[ny/2+2, 2:-2] - H0, 'r--', label="CDKLM")
plt.plot(kp07_h[ny/2+2, 2:-2] - H0, 'g:', label="KP07")
plt.ylim((-0.2, 0.3))
plt.legend()


In [ ]:
## Check conservation of total mass
print("CDKLM:")
print("Initial water volume: ", sum(sum(cdklm_h0[2:-2, 2:-2])))
print("After simulation:     ", sum(sum(cdklm_h[2:-2, 2:-2])))
print("KP07:")
print("Initial water volume: ", sum(sum(kp07_h0[2:-2, 2:-2])))
print("After simulation:     ", sum(sum(kp07_h[2:-2, 2:-2])))
print("CTCS:")
print("Initial water volume: ", sum(sum(ctcs_eta0[1:-1, 1:-1] + H0)))
print("After simulation:     ", sum(sum(ctcs_eta[1:-1, 1:-1] + H0)))


In [ ]:
import gc
fig = None
gc.collect()